In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 21: RPG Simulator 20XX ---</h2><p>Little <span title="The sky above the battle is the color of television, tuned to a dead channel.">Henry Case</span> got a new video game for Christmas.  It's an <a href="https://en.wikipedia.org/wiki/Role-playing_video_game">RPG</a>, and he's stuck on a boss.  He needs to know what equipment to buy at the shop.  He hands you the <a href="https://en.wikipedia.org/wiki/Game_controller">controller</a>.</p>
<p>In this game, the player (you) and the enemy (the boss) take turns attacking.  The player always goes first.  Each attack reduces the opponent's hit points by at least <code>1</code>.  The first character at or below <code>0</code> hit points loses.</p>
<p>Damage dealt by an attacker each turn is equal to the attacker's damage score minus the defender's armor score.  An attacker always does at least <code>1</code> damage.  So, if the attacker has a damage score of <code>8</code>, and the defender has an armor score of <code>3</code>, the defender loses <code>5</code> hit points.  If the defender had an armor score of <code>300</code>, the defender would still lose <code>1</code> hit point.</p>
<p>Your damage score and armor score both start at zero.  They can be increased by buying items in exchange for gold.  You start with no items and have as much gold as you need.  Your total damage or armor is equal to the sum of those stats from all of your items.  You have <em>100 hit points</em>.</p>
<p>Here is what the item shop is selling:</p>
<pre><code>Weapons:    Cost  Damage  Armor
Dagger        8     4       0
Shortsword   10     5       0
Warhammer    25     6       0
Longsword    40     7       0
Greataxe     74     8       0

Armor: Cost Damage Armor
Leather 13 0 1
Chainmail 31 0 2
Splintmail 53 0 3
Bandedmail 75 0 4
Platemail 102 0 5

Rings: Cost Damage Armor
Damage +1 25 1 0
Damage +2 50 2 0
Damage +3 100 3 0
Defense +1 20 0 1
Defense +2 40 0 2
Defense +3 80 0 3
</code></pre>

<p>You must buy exactly one weapon; no dual-wielding.  Armor is optional, but you can't use more than one.  You can buy 0-2 rings (at most one for each hand).  You must use any items you buy.  The shop only has one of each item, so you can't buy, for example, two rings of Damage +3.</p>
<p>For example, suppose you have <code>8</code> hit points, <code>5</code> damage, and <code>5</code> armor, and that the boss has <code>12</code> hit points, <code>7</code> damage, and <code>2</code> armor:</p>
<ul>
<li>The player deals <code>5-2 = 3</code> damage; the boss goes down to 9 hit points.</li>
<li>The boss deals <code>7-5 = 2</code> damage; the player goes down to 6 hit points.</li>
<li>The player deals <code>5-2 = 3</code> damage; the boss goes down to 6 hit points.</li>
<li>The boss deals <code>7-5 = 2</code> damage; the player goes down to 4 hit points.</li>
<li>The player deals <code>5-2 = 3</code> damage; the boss goes down to 3 hit points.</li>
<li>The boss deals <code>7-5 = 2</code> damage; the player goes down to 2 hit points.</li>
<li>The player deals <code>5-2 = 3</code> damage; the boss goes down to 0 hit points.</li>
</ul>
<p>In this scenario, the player wins!  (Barely.)</p>
<p>You have <em>100 hit points</em>.  The boss's actual stats are in your puzzle input.  What is <em>the least amount of gold you can spend</em> and still win the fight?</p>
</article>


In [2]:
from copy import deepcopy
from itertools import combinations
from typing import Generator

from more_itertools import powerset


shop_str = """
Weapons:    Cost  Damage  Armor
Dagger        8     4       0
Shortsword   10     5       0
Warhammer    25     6       0
Longsword    40     7       0
Greataxe     74     8       0

Armor:      Cost  Damage  Armor
Leather      13     0       1
Chainmail    31     0       2
Splintmail   53     0       3
Bandedmail   75     0       4
Platemail   102     0       5

Rings:      Cost  Damage  Armor
Damage +1    25     1       0
Damage +2    50     2       0
Damage +3   100     3       0
Defense +1   20     0       1
Defense +2   40     0       2
Defense +3   80     0       3
"""


@dataclass(frozen=True, order=True)
class ShopItem:
    name: str
    cost: int
    damage: int
    armor: int


class Shop:
    blank_line_regex = re.compile(r"(?:\r?\n){2,}")
    white_spaces = re.compile(r"\s\s+")

    def __init__(self, shop_str: str) -> None:
        weapons, armor, rings = re.split(self.blank_line_regex, shop_str)
        self.weapons = self._parse_category(weapons.strip())
        self.armor = self._parse_category(armor.strip())
        self.rings = self._parse_category(rings.strip())

    @classmethod
    def _parse_category(cls, category_str: str) -> list[ShopItem]:
        return [
            ShopItem(
                *[
                    int(s) if s.isdigit() else s
                    for s in re.split(cls.white_spaces, line)
                ]
            )
            for line in category_str.splitlines()[1:]
        ]

    def __repr__(self) -> str:
        return f'''
Shop:
    Weapons: 
        {'\n\t'.join(f'{si}' for si in self.weapons)}
    Armor: 
        {'\n\t'.join(f'{si}' for si in self.armor)}
    Rings: 
        {'\n\t'.join(f'{si}' for si in self.rings)}
'''


@dataclass
class Player:
    hit_points: int
    damage: int
    armor: int


class Game:
    def __init__(self, shop_str: str, player: Player, enemy: Player) -> None:
        self.shop = Shop(shop_str)
        self.player = player
        self.enemy = enemy

    def scenario(self, do_print: bool = False) -> bool:

        while True:
            damage = max(self.player.damage - self.enemy.armor, 1)
            self.enemy.hit_points -= damage

            if do_print:
                print(
                    f"The player deals {self.player.damage}-{self.enemy.armor} = {damage}; The boss goes down to {self.enemy.hit_points} hit points"
                )

            if self.enemy.hit_points <= 0:
                return True

            damage = max(self.enemy.damage - self.player.armor, 1)
            self.player.hit_points -= damage

            if do_print:
                print(
                    f"The boss deals {self.enemy.damage}-{self.player.armor} = {damage}; The player goes down to {self.player.hit_points} hit points"
                )

            if self.player.hit_points <= 0:
                return False

    def generate(self) -> Generator[tuple[ShopItem, list[ShopItem], list[ShopItem]]]:
        # You must buy exactly one weapon.
        # Armor is optional, but you can't use more than one.
        # You can buy 0-2 rings
        for weapon in self.shop.weapons:
            yield [weapon, [], []]
            for armor in self.shop.armor:
                yield [weapon, [armor], []]
                for ring in self.shop.rings:
                    yield [weapon, [], [ring]]
                    yield [weapon, [armor], [ring]]
                for rings in combinations(self.shop.rings, 2):
                    yield [weapon, [], rings]
                    yield [weapon, [armor], rings]

    def cheapest_winning(self) -> int:
        min_cost = inf
        enemy = deepcopy(self.enemy)
        hit_points = self.player.hit_points
        for weapon, armor, rings in self.generate():
            self.enemy = deepcopy(enemy)
            self.player.hit_points = hit_points

            self.player.armor = (
                weapon.armor
                + sum((a.armor for a in armor), start=0)
                + sum((r.armor for r in rings), start=0)
            )
            self.player.damage = (
                weapon.damage
                + sum((a.damage for a in armor), start=0)
                + sum((r.damage for r in rings), start=0)
            )

            if self.scenario():
                cost = (
                    weapon.cost
                    + sum((a.cost for a in armor), start=0)
                    + sum((r.cost for r in rings), start=0)
                )
                if cost < min_cost:
                    min_cost = cost

        return min_cost

    def most_expensive_losing(self) -> int:
        max_cost = -inf
        enemy = deepcopy(self.enemy)
        hit_points = self.player.hit_points

        for weapon, armor, rings in self.generate():
            self.enemy = deepcopy(enemy)

            self.player.hit_points = hit_points

            self.player.armor = (
                weapon.armor
                + sum((a.armor for a in armor), start=0)
                + sum((r.armor for r in rings), start=0)
            )
            self.player.damage = (
                weapon.damage
                + sum((a.damage for a in armor), start=0)
                + sum((r.damage for r in rings), start=0)
            )

            if not self.scenario():
                cost = (
                    weapon.cost
                    + sum((a.cost for a in armor), start=0)
                    + sum((r.cost for r in rings), start=0)
                )
                if cost > max_cost:
                    max_cost = cost

        return max_cost


game = Game(shop_str, Player(8, 5, 5), Player(12, 7, 2))


game.scenario(True)

The player deals 5-2 = 3; The boss goes down to 9 hit points
The boss deals 7-5 = 2; The player goes down to 6 hit points
The player deals 5-2 = 3; The boss goes down to 6 hit points
The boss deals 7-5 = 2; The player goes down to 4 hit points
The player deals 5-2 = 3; The boss goes down to 3 hit points
The boss deals 7-5 = 2; The player goes down to 2 hit points
The player deals 5-2 = 3; The boss goes down to 0 hit points


True

In [3]:
# Hit Points: 104
# Damage: 8
# Armor: 1
boss = Player(104, 8, 1)
me = Player(100, 0, 0)

game = Game(shop_str, me, boss)
game.cheapest_winning()

78

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>78</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Turns out the shopkeeper is working with the boss, and can persuade you to buy whatever items he wants. The other rules still apply, and he still only has one of each item.</p>
<p>What is the <em>most</em> amount of gold you can spend and still <em>lose</em> the fight?</p>
</article>

</main>


In [4]:
# Hit Points: 104
# Damage: 8
# Armor: 1
boss = Player(104, 8, 1)
me = Player(100, 0, 0)

game = Game(shop_str, me, boss)
game.most_expensive_losing()
# 141 too low and 464 too high

148

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>148</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2015">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="21/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22RPG+Simulator+20XX%22+%2D+Day+21+%2D+Advent+of+Code+2015&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F21&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22RPG+Simulator+20XX%22+%2D+Day+21+%2D+Advent+of+Code+2015+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2015%2Fday%2F21';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
